In [ ]:
import os
import re
import json
import numpy as np
import pandas as pd

In [ ]:
model_path = r"C:\Users\user\Desktop\temp\rolevectors_results\Qwen-7B-Chat"

In [ ]:
ROLE_DATASET_MAPPING = {
    "econ": ["economic researcher", "economist", "financial analyst"],
    "eecs": ["electronics technician", "data scientist", "electrical engineer", "software engineer", "web developer"],
    "law": ["bailiff", "lawyer"],
    "math": ["data analyst", "mathematician", "statistician"],
    "medicine": ["nurse", "doctor", "physician", "dentist", "surgeon"],
    "natural science": ["geneticist", "biologist", "physicist", "teacher", "chemist", "ecologist"],
    "politics": ["politician", "sheriff", "governor", "enthusiast", "partisan"],
    "psychology": ["psychologist"]
}

In [ ]:

def format_baseline(df):
    # Create a copy to avoid modifying the original DataFrame
    df_formatted = df.copy()
    
    # For each dataset in the MultiIndex (first level of columns)
    for dataset in df.columns.levels[0]:
        if 'baseline' in df[dataset].columns:
            # Instead of rounding, simply convert to string.
            df_formatted[(dataset, 'baseline')] = df[dataset]['baseline'].apply(lambda x: str(x))
    
    return df_formatted

def latex_color_columns(df):
    """
    Returns strings that include LaTeX commands (e.g. \cellcolor from the xcolor package)
    and wraps numerical values in math mode. For columns representing percentual increments 
    (e.g., "1.0", "3.0", "ablation"), the cell is colored only if the row’s role (assumed to be
    the row index) is associated with the dataset according to ROLE_DATASET_MAPPING.
    """
    
    def latex_color_value(val, _baseline):
        """
        Formats a value (which is already the percentual increment) with a LaTeX cell color command.
        Positive increments are colored in green, negative in red, with intensity based on the magnitude.
        """
        try:
            if isinstance(val, str) and '±' in val:
                # Extract the main value, ignoring the uncertainty part
                val = float(val.split('±')[0].strip())
            else:
                val = float(val)
        except Exception:
            val = 0.0

        # Round to one decimal; here, val represents the percentual increment.
        val = round(val, 1)
        
        # Compute a color command only if there is an increment.
        if val == 0:
            color_cmd = ""
        else:
            # Logarithmic scaling to determine intensity.
            intensity_value = np.log1p(abs(val))  # log(1 + |percentage increment|)
            factor = 10  # scaling factor (adjust as needed)
            min_pct = 10
            max_pct = 50
            pct = int(np.clip(intensity_value * factor, min_pct, max_pct))
            color_cmd = f"\\cellcolor{{green!{pct}}}" if val > 0 else f"\\cellcolor{{red!{pct}}}"
        
        # Force a plus sign for non-negative values.
        if val >= 0:
            formatted_val = f"+{val:.1f}\\%"
        else:
            formatted_val = f"{val:.1f}\\%"
        
        return f"{color_cmd} $ {formatted_val} $"
    
    def plain_format_value(val):
        """
        Formats the value without applying any cell color command.
        """
        try:
            if isinstance(val, str) and '±' in val:
                val = float(val.split('±')[0].strip())
            else:
                val = float(val)
        except Exception:
            val = 0.0
        val = round(val, 1)
        formatted_val = f"+{val:.1f}\\%" if val >= 0 else f"{val:.1f}\\%"
        return f"$ {formatted_val} $"
    
    def latex_baseline(val):
        # Wrap the baseline value in math mode (preserving its original precision)
        return f"$ {val} $"
    
    def style_df(x):
        # Create an empty DataFrame for the LaTeX-formatted cell contents.
        df_styled = pd.DataFrame('', index=x.index, columns=x.columns)
        
        # Iterate over datasets (first level of columns).
        for dataset in x.columns.levels[0]:
            if 'baseline' in x[dataset].columns:
                baseline_values = x[dataset]['baseline']
                for idx in x.index:
                    # Always format the baseline column.
                    df_styled.loc[idx, (dataset, 'baseline')] = latex_baseline(x.loc[idx, (dataset, 'baseline')])
                    # Assume the row index is the role (case-insensitive).
                    role = str(idx).lower()
                    allowed_roles = [r.lower() for r in ROLE_DATASET_MAPPING.get(dataset, [])]
                    
                    # Process other columns (e.g., "1.0", "3.0", "ablation").
                    for col in x[dataset].columns:
                        if col == 'baseline':
                            continue
                        value = x.loc[idx, (dataset, col)]
                        if role in allowed_roles:
                            df_styled.loc[idx, (dataset, col)] = latex_color_value(value, baseline_values[idx])
                        else:
                            df_styled.loc[idx, (dataset, col)] = plain_format_value(value)
        return df_styled
    
    return style_df(df)


In [ ]:

# Mapping from dataset category to list of roles.
ROLE_DATASET_MAPPING = {
    "econ": ["economic researcher", "economist", "financial analyst"],
    "eecs": ["electronics technician", "data scientist", "electrical engineer", "software engineer", "web developer"],
    "law": ["bailiff", "lawyer"],
    "math": ["data analyst", "mathematician", "statistician"],
    "medicine": ["nurse", "doctor", "physician", "dentist", "surgeon"],
    "natural_science": ["geneticist", "biologist", "physicist", "teacher", "chemist", "ecologist"],
    "politics": ["politician", "sheriff", "governor", "enthusiast", "partisan"],
    "psychology": ["psychologist"]
}

def get_dataset_category(role):
    """
    Returns the dataset category for a given role by looking up the mapping.
    """
    for category, roles in ROLE_DATASET_MAPPING.items():
        if role in roles:
            return category
    return None

def compute_mean_score(filepath):
    """
    Given a JSON file (a list of dicts), compute the mean of the 'score' values.
    """
    try:
        with open(filepath, "r") as f:
            data = json.load(f)
        scores = [d["score"] for d in data if "score" in d]
        return round(np.mean(scores), 2) if scores else np.nan
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return np.nan

def get_baseline(select_dir):
    """
    In the run version folder (e.g. "1.0/select_direction"),
    load results_baseline.json and return its mean score.
    """
    baseline_file = os.path.join(select_dir, "results_baseline.json")
    if os.path.exists(baseline_file):
        return compute_mean_score(baseline_file)
    else:
        return np.nan

def get_addition_scores(select_dir, layer_range=False, start_percent=50):
    """
    Iterate over coefficient subdirectories under select_dir and compute scores.
    """
    addition_scores = []
    for sub in os.listdir(select_dir):
        sub_path = os.path.join(select_dir, sub)
        if os.path.isdir(sub_path) and re.match(r"^-?\d+$", sub):
            inner_dirs = [
                d for d in os.listdir(sub_path)
                if os.path.isdir(os.path.join(sub_path, d)) and re.match(r"^\d+$", d)
            ]
            inner_dirs_sorted = sorted(inner_dirs, key=lambda x: int(x))
            n_layers = len(inner_dirs_sorted)
            if n_layers == 0:
                continue
            if layer_range:
                start_idx = int(np.floor(n_layers * (start_percent / 100)))
                end_idx = int(np.ceil(n_layers * 0.8))
                start_idx = min(start_idx, end_idx - 1)
                selected_dirs = inner_dirs_sorted[start_idx:end_idx]
            else:
                keep_count = max(1, int(np.ceil(n_layers * 0.8)))
                selected_dirs = inner_dirs_sorted[:keep_count]
            
            for inner in selected_dirs:
                inner_path = os.path.join(sub_path, inner)
                for file in os.listdir(inner_path):
                    if file.startswith("results_addition") and file.endswith(".json"):
                        file_path = os.path.join(inner_path, file)
                        mean_file_score = compute_mean_score(file_path)
                        if not np.isnan(mean_file_score):
                            addition_scores.append(mean_file_score)
    
    if addition_scores:
        return np.mean(addition_scores), np.std(addition_scores)
    else:
        return np.nan, np.nan

def get_select_best_addition_score(version_path):
    """
    Get addition score for select_best mode by reading the metadata file.
    The metadata is expected at:
      {model}/{role}/{dataset}/{coeff}/direction_metadata.json
    Then, the addition score is computed from:
      {model}/{role}/{dataset}/{coeff}/select_direction/{pos}/{layer}/results_addition_{pos}_{layer}.json
    """
    metadata_path = os.path.join(version_path, "direction_metadata.json")
    if not os.path.exists(metadata_path):
        print(f"Warning: metadata file not found in {metadata_path}")
        return np.nan, np.nan
    try:
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    except Exception as e:
        print(f"Error reading metadata file {metadata_path}: {e}")
        return np.nan, np.nan
    
    if "pos" not in metadata or "layer" not in metadata:
        print(f"Warning: metadata file {metadata_path} missing 'pos' or 'layer'")
        return np.nan, np.nan
    
    pos = str(metadata["pos"])
    layer = str(metadata["layer"])
    select_dir = os.path.join(version_path, "select_direction")
    addition_file = os.path.join(select_dir, pos, layer, f"results_addition_{pos}_{layer}.json")
    if not os.path.exists(addition_file):
        print(f"Warning: addition file not found: {addition_file}")
        return np.nan, np.nan
    score = compute_mean_score(addition_file)
    # Since this is a single performance, we return the mean only (std remains NaN)
    return score, np.nan

def process_run_version(version_path, layer_range=False, start_percent=50, select_best=False):
    """
    Process the run version and compute baseline and addition scores.
    If select_best is True, the metadata is used to select the single best direction.
    """
    select_dir = os.path.join(version_path, "select_direction")
    if not os.path.exists(select_dir):
        return (np.nan, (np.nan, np.nan))
    
    # Only version "1.0" is expected to have a baseline.
    base_val = get_baseline(select_dir) if os.path.basename(version_path) == "1.0" else np.nan
    
    if select_best:
        addition_mean, addition_std = get_select_best_addition_score(version_path)
    else:
        addition_mean, addition_std = get_addition_scores(select_dir, layer_range, start_percent)
    
    return base_val, (addition_mean, addition_std)

def get_best_direction(role, primary_dataset_path, version, baseline):
    """
    For a given role and version folder inside the primary dataset folder, load the filtered direction evaluations file
    and return (position, layer) as strings.

    Expects the JSON file at:
       {model}/{role}/{primary_dataset}/{version}/select_direction/direction_evaluations_filtered.json

    When version is "3.0", since the file in that folder doesn't contain valid ablation information,
    the ablation values are retrieved from the baseline file at:
       {model}/{role}/{primary_dataset}/1.0/select_direction/direction_evaluations_filtered.json

    For versions other than "3.0", the JSON file is expected to contain both steering and ablation values.
    
    The function first searches for an entry where:
       - steering_performance_score >= baseline, and
       - ablation_performance_score < baseline.
       
    If no such entry is found:
       - For version "3.0": the function selects the candidate with the highest steering_performance_score from the 3.0 file.
       - For version "1.0": the function relaxes the ablation condition and selects the candidate with the highest steering_performance_score.
    
    If a suitable entry is found, its "position" and "layer" values are returned as strings.
    Otherwise, (None, None) is returned.
    """
    if version == "3.0":
        eval_file_path = os.path.join(
            primary_dataset_path, version, "select_direction", "direction_evaluations_filtered.json"
        )
        # Load ablation values from baseline (version "1.0")
        baseline_eval_file_path = os.path.join(
            primary_dataset_path, "1.0", "select_direction", "direction_evaluations_filtered.json"
        )
        
        if not os.path.exists(eval_file_path):
            print(f"Warning: filtered evaluations file not found for role '{role}' in {eval_file_path}")
            return None, None
        if not os.path.exists(baseline_eval_file_path):
            print(f"Warning: baseline filtered evaluations file not found for role '{role}' in {baseline_eval_file_path}")
            return None, None
        
        try:
            with open(eval_file_path, "r") as f:
                eval_entries = json.load(f)
            with open(baseline_eval_file_path, "r") as f:
                baseline_entries = json.load(f)
            
            # Build a dictionary mapping (position, layer) to ablation_performance_score from the baseline file.
            baseline_ablation = {}
            for entry in baseline_entries:
                if ("position" in entry and "layer" in entry and 
                    "ablation_performance_score" in entry):
                    key = (str(entry["position"]), str(entry["layer"]))
                    baseline_ablation[key] = entry["ablation_performance_score"]
            
            # First, try to find an entry satisfying the conditions.
            for entry in eval_entries:
                if ("steering_performance_score" in entry and 
                    "position" in entry and 
                    "layer" in entry):
                    pos = str(entry["position"])
                    layer = str(entry["layer"])
                    steer_perf = entry["steering_performance_score"]
                    
                    key = (pos, layer)
                    if key not in baseline_ablation:
                        continue
                    ablation_perf = baseline_ablation[key]
                    
                    if steer_perf >= baseline and ablation_perf < baseline:
                        return pos, layer
            
            # If no candidate meets the conditions, select the best candidate from the 3.0 evaluations
            best_entry = None
            best_score = -float("inf")
            for entry in eval_entries:
                if ("steering_performance_score" in entry and 
                    "position" in entry and 
                    "layer" in entry):
                    score = entry["steering_performance_score"]
                    if score > best_score:
                        best_score = score
                        best_entry = entry
            
            if best_entry is not None:
                return str(best_entry["position"]), str(best_entry["layer"])
            else:
                print(f"Warning: No valid entries found in {eval_file_path}.")
                return None, None
        except Exception as e:
            print(f"Error reading evaluations for role '{role}' in version '3.0': {e}")
            return None, None

    else:
        # For versions other than "3.0"
        eval_file_path = os.path.join(
            primary_dataset_path, version, "select_direction", "direction_evaluations_filtered.json"
        )
        
        if not os.path.exists(eval_file_path):
            print(f"Warning: filtered evaluations file not found for role '{role}' in {eval_file_path}")
            return None, None

        try:
            with open(eval_file_path, "r") as f:
                evaluations = json.load(f)
                
            # First, try to find an entry satisfying the conditions.
            for entry in evaluations:
                if ("steering_performance_score" in entry and 
                    "ablation_performance_score" in entry and
                    "position" in entry and 
                    "layer" in entry):
                    
                    if (entry["steering_performance_score"] >= baseline and 
                        entry["ablation_performance_score"] < baseline):
                        return str(entry["position"]), str(entry["layer"])
            
            # For version "1.0", relax the ablation requirement if no candidate is found.
            if version == "1.0":
                best_entry = None
                best_score = -float("inf")
                for entry in evaluations:
                    if ("steering_performance_score" in entry and 
                        "position" in entry and 
                        "layer" in entry):
                        score = entry["steering_performance_score"]
                        if score > best_score:
                            best_score = score
                            best_entry = entry
                if best_entry is not None:
                    return str(best_entry["position"]), str(best_entry["layer"])
            
            print(f"Warning: No evaluation entry in {eval_file_path} satisfies the conditions.")
            return None, None
        except Exception as e:
            print(f"Error reading evaluations for role '{role}' in {eval_file_path}: {e}")
            return None, None




def get_score_for_dataset_with_direction(model_name, role, dataset, version, pos, layer):
    """
    For a given role, dataset, and version (coefficient) folder, load the results file using the provided
    best direction parameters, compute the new score from:
       {model_name}/{role}/{dataset}/{version}/select_direction/{pos}/{layer}/results_addition_{pos}_{layer}.json
    Then, load the baseline score from:
       {model_name}/{role}/{dataset}/1.0/select_direction/results_baseline.json
    and return the percentual increment (with sign) as a formatted string:
       (new_score - baseline) / baseline * 100
    If any step fails, "nan" is returned.
    """
    file_path = os.path.join(
        model_name, role, dataset, version, "select_direction", pos, layer,
        f"results_addition_{pos}_{layer}.json"
    )
    if not os.path.exists(file_path):
        print(f"Warning: score file not found: {file_path}")
        return "nan"
    
    new_score = compute_mean_score(file_path)
    
    # Load the baseline score from version "1.0".
    baseline_dir = os.path.join(model_name, role, dataset, "1.0", "select_direction")
    baseline_score = get_baseline(baseline_dir)
    
    if np.isnan(new_score) or np.isnan(baseline_score) or baseline_score == 0:
        return "nan"
    
    increment = (new_score - baseline_score) / baseline_score * 100
    return f"{increment:+.1f}"


def get_score_for_dataset_with_direction_ablation(model_name, role, dataset, version, pos, layer):
    """
    For a given role, dataset, and version (coefficient) folder, load the results file using the provided
    best direction parameters, compute the new score from:
       {model_name}/{role}/{dataset}/{version}/select_direction/{pos}/{layer}/results_ablation_{pos}_{layer}.json
    Then, load the baseline score from:
       {model_name}/{role}/{dataset}/1.0/select_direction/results_baseline.json
    and return the percentual increment (with sign) as a formatted string:
       (new_score - baseline) / baseline * 100
    If any step fails, "nan" is returned.
    """
    file_path = os.path.join(
        model_name, role, dataset, version, "select_direction", pos, layer,
        f"results_ablation_{pos}_{layer}.json"
    )
    if not os.path.exists(file_path):
        print(f"Warning: score file not found: {file_path}")
        return "nan"
    
    new_score = compute_mean_score(file_path)
    
    # Load the baseline score from version "1.0".
    baseline_dir = os.path.join(model_name, role, dataset, "1.0", "select_direction")
    baseline_score = get_baseline(baseline_dir)
    
    if np.isnan(new_score) or np.isnan(baseline_score) or baseline_score == 0:
        return "nan"
    
    increment = (new_score - baseline_score) / baseline_score * 100
    return f"{increment:+.1f}"


def main(model_name, layer_range=False, start_percent=50, select_best=False, select_best_role=False, select_best_role_ablation=False):
    """
    Builds a MultiIndex DataFrame reporting performance scores.
    
    When select_best_role is True, for each role the code:
      1. Determines the "primary" dataset via get_dataset_category().
      2. In that primary folder (e.g. {model}/{role}/{primary_dataset}) identifies all version folders.
      3. For each version, loads the best direction from its filtered evaluations file using the provided baseline.
      4. Then for every dataset folder under {model}/{role} (excluding folders like "generate_directions" or "test_direction"),
         if that folder contains the same version folder, the score is loaded from:
              {model}/{role}/{dataset}/{version}/select_direction/{pos}/{layer}/results_addition_{pos}_{layer}.json
      5. For version "1.0", the score is also recorded as the baseline.
    
    The DataFrame’s rows are roles and its columns use a MultiIndex with:
      - Level 0: dataset folder name (e.g. "law", "math", …)
      - Level 1: version (with "baseline" included for version "1.0")
    
    (When select_best_role is False, the original logic is used; that branch is not implemented here.)
    """
    if sum([bool(layer_range), bool(select_best), bool(select_best_role), bool(select_best_role_ablation)]) > 1:
         raise ValueError("Only one of layer_range, select_best, select_best_role, or select_best_role_ablation can be True.")
    
    if select_best_role:
         roles = [d for d in os.listdir(model_name) if os.path.isdir(os.path.join(model_name, d))]
         results = {}
         for role in roles:
              role_path = os.path.join(model_name, role)
              primary_dataset = get_dataset_category(role)
              if primary_dataset is None:
                  print(f"Warning: No primary dataset mapping for role '{role}'. Skipping best-direction evaluation for this role.")
                  continue
              primary_dataset_path = os.path.join(role_path, primary_dataset)
              if not os.path.exists(primary_dataset_path):
                  print(f"Warning: Primary dataset folder '{primary_dataset_path}' not found for role '{role}'. Skipping.")
                  continue
              # Compute baseline from the primary dataset's version "1.0" select_direction folder.
              baseline_select_dir = os.path.join(primary_dataset_path, "1.0", "select_direction")
              baseline_value = get_baseline(baseline_select_dir)
              # Get the list of version folders (e.g. "1.0", "3.0", etc.) from the primary dataset folder.
              primary_versions = [v for v in os.listdir(primary_dataset_path) if re.match(r"^\d+\.\d+$", v)]
              primary_versions = sorted(primary_versions, key=lambda x: float(x))
              # Get all dataset directories under the role (excluding known non-performance folders).
              all_datasets = [d for d in os.listdir(role_path) 
                              if os.path.isdir(os.path.join(role_path, d)) 
                              and d not in ["generate_directions", "test_direction"]]
              # Prepare a dictionary: for each dataset, a dict mapping version (and baseline) to score.
              ds_result = {ds: {} for ds in all_datasets}
              for ver in primary_versions:
                   # Get best direction using the new function that requires baseline.
                   pos, layer = get_best_direction(role, primary_dataset_path, ver, baseline_value)
                   if pos is None or layer is None:
                        print(f"Skipping version {ver} for role '{role}' due to missing best direction.")
                        continue
                   # For each dataset folder, if it contains the same version folder, load its score.
                   for ds in all_datasets:
                        ds_ver_path = os.path.join(role_path, ds, ver)
                        if os.path.exists(ds_ver_path):
                            score = get_score_for_dataset_with_direction(model_name, role, ds, ver, pos, layer)
                            ds_result[ds][ver] = score if score != "nan" else "nan"
                        else:
                            ds_result[ds][ver] = "nan"
                        # If this is the "1.0" version, also record its baseline.
                        if ver == "1.0":
                            select_dir = os.path.join(model_name, role, ds, ver, "select_direction")
                            baseline_score = get_baseline(select_dir)
                            ds_result[ds]["baseline"] = f"{baseline_score:.2f}" if not np.isnan(baseline_score) else "nan"
              results[role] = ds_result
         
         # Build a DataFrame from the results dictionary.
         all_dataset_set = set()
         all_version_set = set()
         for role, ds_dict in results.items():
              for ds, ver_dict in ds_dict.items():
                   all_dataset_set.add(ds)
                   all_version_set.update(ver_dict.keys())
         all_dataset_list = sorted(all_dataset_set)
         # Sort versions so that "baseline" always comes first, then numeric order.
         def sort_ver(x):
              if x == "baseline":
                   return (0, 0)
              try:
                   return (1, float(x))
              except:
                   return (1, x)
         all_version_list = sorted(list(all_version_set), key=sort_ver)
         cols = pd.MultiIndex.from_product([all_dataset_list, all_version_list], names=["Dataset", "Version"])
         df = pd.DataFrame(index=sorted(results.keys()), columns=cols)
         for role, ds_dict in results.items():
              for ds, ver_dict in ds_dict.items():
                   for ver, value in ver_dict.items():
                        df.loc[role, (ds, ver)] = value
         return df

    elif select_best_role_ablation:
         roles = [d for d in os.listdir(model_name) if os.path.isdir(os.path.join(model_name, d))]
         results = {}
         for role in roles:
              role_path = os.path.join(model_name, role)
              primary_dataset = get_dataset_category(role)
              if primary_dataset is None:
                  print(f"Warning: No primary dataset mapping for role '{role}'. Skipping best-direction evaluation for this role.")
                  continue
              primary_dataset_path = os.path.join(role_path, primary_dataset)
              if not os.path.exists(primary_dataset_path):
                  print(f"Warning: Primary dataset folder '{primary_dataset_path}' not found for role '{role}'. Skipping.")
                  continue
              # Compute baseline from the primary dataset's version "1.0" select_direction folder.
              baseline_select_dir = os.path.join(primary_dataset_path, "1.0", "select_direction")
              baseline_value = get_baseline(baseline_select_dir)
              # Get the list of version folders from the primary dataset folder.
              primary_versions = [v for v in os.listdir(primary_dataset_path) if re.match(r"^\d+\.\d+$", v)]
              primary_versions = sorted(primary_versions, key=lambda x: float(x))
              # Get all dataset directories under the role (excluding known non-performance folders).
              all_datasets = [d for d in os.listdir(role_path) 
                              if os.path.isdir(os.path.join(role_path, d)) 
                              and d not in ["generate_directions", "test_direction"]]
              # Prepare a dictionary: for each dataset, a dict mapping version (and baseline) to score.
              ds_result = {ds: {} for ds in all_datasets}
              for ver in primary_versions:
                   if ver == "3.0":
                       continue
                   # Get best direction using baseline_value.
                   pos, layer = get_best_direction(role, primary_dataset_path, ver, baseline_value)
                   if pos is None or layer is None:
                        print(f"Skipping version {ver} for role '{role}' due to missing best direction.")
                        continue
                   # For each dataset folder, if it contains the same version folder, load its ablation score.
                   for ds in all_datasets:
                        ds_ver_path = os.path.join(role_path, ds, ver)
                        if os.path.exists(ds_ver_path):
                            score = get_score_for_dataset_with_direction_ablation(model_name, role, ds, ver, pos, layer)
                            ds_result[ds]["ablation"] = score if score != "nan" else "nan"
                        else:
                            ds_result[ds]["ablation"] = "nan"
                        # If this is the "1.0" version, also record its baseline.
                        if ver == "1.0":
                            select_dir = os.path.join(model_name, role, ds, ver, "select_direction")
                            baseline_score = get_baseline(select_dir)
                            ds_result[ds]["baseline"] = f"{baseline_score:.2f}" if not np.isnan(baseline_score) else "nan"
              results[role] = ds_result
         
         # Build a DataFrame from the results dictionary.
         all_dataset_set = set()
         all_version_set = set()
         for role, ds_dict in results.items():
              for ds, ver_dict in ds_dict.items():
                   all_dataset_set.add(ds)
                   all_version_set.update(ver_dict.keys())
         all_dataset_list = sorted(all_dataset_set)
         # Sort versions so that "baseline" always comes first, then numeric order.
         def sort_ver(x):
              if x == "baseline":
                   return (0, 0)
              try:
                   return (1, float(x))
              except:
                   return (1, x)
         all_version_list = sorted(list(all_version_set), key=sort_ver)
         cols = pd.MultiIndex.from_product([all_dataset_list, all_version_list], names=["Dataset", "Version"])
         df = pd.DataFrame(index=sorted(results.keys()), columns=cols)
         for role, ds_dict in results.items():
              for ds, ver_dict in ds_dict.items():
                   for ver, value in ver_dict.items():
                        df.loc[role, (ds, ver)] = value
         return df

    else:
         raise NotImplementedError("Other modes (non-select_best_role) are not implemented in this snippet.")



In [ ]:
if __name__ == "__main__":
    model_dir = model_path
    df = main(model_dir, select_best_role_ablation=True)
    # Print the DataFrame as LaTeX code
    # Generate the LaTeX table code from the DataFrame
    # Suppose df is your DataFrame with roles as its index.
    # Assume df is your DataFrame with roles as its index.
    desired_group_order = ["econ", "eecs", "law", "math", "medicine", "natural_science", "politics", "psychology"]

    ordered_roles = []
    for group in desired_group_order:
        # Get roles for the group; if some roles are missing in df.index, they won't be added.
        for role in ROLE_DATASET_MAPPING.get(group, []):
            if role in df.index:
                ordered_roles.append(role)

    # Optionally, if there are roles in df.index that weren't included in the mapping,
    # you can append them (here sorted alphabetically, but you can choose another order)
    remaining_roles = [role for role in df.index if role not in ordered_roles]
    ordered_roles.extend(sorted(remaining_roles))

    # Reindex the DataFrame according to the custom order.
    df = df.reindex(ordered_roles)


    # First format the baseline values
    df_formatted = format_baseline(df)

    # Then apply the styling
    df_styled = latex_color_columns(df_formatted)  # This function returns a new DataFrame with LaTeX strings.

In [ ]:
df

In [ ]:
#Save the dataframe to a csv file
#take model name
model_name = model_path.split("\\")[-1]
#save the dataframe to a csv file
df.to_csv(f"{model_name}_ablation.csv")

In [ ]:
print(df_styled.to_latex(escape=False))